<a target="_blank" href="https://colab.research.google.com/github/vnghia/5ma-hddl/blob/main/objectness.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


In [ ]:
!pip install -q git+https://github.com/facebookresearch/detectron2@95a87b8dd359014d1ff81fe14f539dd279bcbe4b

In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
%matplotlib inline
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import BoxMode
from detectron2.modeling.proposal_generator.rpn import StandardRPNHead

In [ ]:
import numpy as np
import os, json, cv2, random, functools
import matplotlib.pyplot as plt

In [ ]:
import torch
import torchvision

In [ ]:
objectness = []
def forward_with_objectness(self, features):
    objectness.clear()
    pred_objectness_logits = []
    pred_anchor_deltas = []
    for x in features:
        t = self.conv(x)
        pred_objectness_logits.append(self.objectness_logits(t))
        pred_anchor_deltas.append(self.anchor_deltas(t))
        o = pred_objectness_logits[-1].sigmoid() * 255
        o = o.cpu().detach().numpy()[0, 1]
        objectness.append(o)
    return pred_objectness_logits, pred_anchor_deltas

In [ ]:
StandardRPNHead.forward = forward_with_objectness 

In [ ]:
cfg = model_zoo.get_config("COCO-Detection/rpn_R_50_FPN_1x.yaml", True)

In [ ]:
predictor = DefaultPredictor(cfg)

In [ ]:
!wget https://www.reidsengland.com/site/assets/files/2315/routemaster-new-st-pauls.jpg -O image.jpg

In [ ]:
im = cv2.imread("./image.jpg")
plt.figure(figsize=(15,7.5))
plt.imshow(im[..., ::-1])

In [ ]:
outputs = predictor(im[..., ::-1])

In [ ]:
import matplotlib.pyplot as plt
import cv2
imgs = []

h_out, w_out, _ = im.shape

imgs.append(cv2.imread('image.jpg'))
for i in range(5):  # five levels
    heatmap = np.asarray(cv2.resize(objectness[i], (w_out, h_out), fx=0, fy=0, interpolation = cv2.INTER_NEAREST), dtype=np.uint8)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    imgs.append(imgs[0] // 2 + heatmap // 2)  # blending
fig = plt.figure(figsize=(16, 7))
for i, img in enumerate(imgs):
    fig.add_subplot(2, 3, i + 1)
    if i > 0:
        plt.imshow(img[0:-1, :, ::-1])  # ::-1 removes the edge
        plt.title("objectness on P" + str(i + 1))
    else:
        plt.imshow(img[:, :, ::-1])
        plt.title("input image")
plt.show()